In [61]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [62]:
mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="" )
print(mydb)

In [64]:
mycursor=mydb.cursor()
mycursor.execute("create database Authorized_user")

In [65]:
mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="Authorized_user")
mycursor=mydb.cursor()

mycursor.execute("create table my_table(id int primary key, Name varchar(50), Reg_no int , branch varchar(50))")

In [77]:
window=tk.Tk()
window.title("Face recognition system")
#window.config(background="lime")
l4 = tk.Label(window, text="Fill The Following Details : ", font=("times new roman",15,"underline","bold"))
l4.place(x=0,y=0)
l1 = tk.Label(window, text="Name: ", font=("Arial",15))
l1.place(x=0,y=40)
t1 = tk.Entry(window, bd=5, width=50)
t1.place(x=90, y=40)

l2 = tk.Label(window, text="Reg. No: ", font=("Arial",15))
l2.place(x=0,y=80)
t2 = tk.Entry(window, bd=5, width=50)
t2.place(x=90, y=80)

l3 = tk.Label(window, text="Branch: ", font=("Arial",15))
l3.place(x=0,y=120)
t3 = tk.Entry(window, bd=5, width=50)
t3.place(x=90, y=120)

l5 = tk.Label(window, text="Execution:  ", font=("times new roman",15,"underline","bold"))
l5.place(x=0,y=180)


    
def generate_dataset():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please provide complete details of the user')
    else:
        mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="Authorized_user"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into my_table(id,Name,Reg_no,branch) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray,1.3,5)
            #scaling factor=1.3
            #Minimum neighbor = 5

            if faces is ():
                return None
            for(x,y,w,h) in faces:
                cropped_face=img[y:y+h,x:x+w]
            return cropped_face

        cap = cv2.VideoCapture(0)
        img_id=0

        while True:
            ret,frame = cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face = cv2.resize(face_cropped(frame),(200,200))
                face  = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path,face)
                cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1, (0,255,0),2)
                # (50,50) is the origin point from where text is to be written
                # font scale=1
                #thickness=2

                cv2.imshow("Cropped face",face)
                if cv2.waitKey(1)==13 or int(img_id)==200:
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','Generating dataset completed!!!')
b1 = tk.Button(window, text="1.Create dataset",anchor="w",font=("times new roman",20),bg="yellow",fg="black", width="20",command=generate_dataset)
b1.place(x=0,y=220)


def train_classifier():
    data_dir="C:/Users/RITIK RAUNAK/AppData/Local/Programs/Python/Python310/Scripts/New folder/data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces  = []
    ids   = []
    
    for image in path:
        img = Image.open(image).convert('L');
        imageNp= np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    
    #Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result','Training dataset completed!!!')
b2 = tk.Button(window, text="2.Train the model",anchor="w", font=("times new roman",20), bg="skyblue", fg="black",width="20",command=train_classifier)
b2.place(x=0,y=280)

def detect_face():
    def draw_boundary(img,classifier,scaleFactor,minNeighbors,color,text,clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)

        coords = []

        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            id,pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            mydb=mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="Authorized_user"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select name from my_table where id="+str(id))
            s = mycursor.fetchone()
            s = ''+''.join(s)
            
            if confidence>74:
                cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)   
            else:
                cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)

            coords=[x,y,w,h]
        return coords
            
    def recognize(img,clf,faceCascade):
        coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img

    faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture =  cv2.VideoCapture(0)

    while True:
        ret,img = video_capture.read()
        img=  recognize(img,clf,faceCascade)
        
        cv2.imshow("face detection",img)

        if cv2.waitKey(1)==13:
            break

    video_capture.release()
    cv2.destroyAllWindows()
b3 = tk.Button(window, text="3.Detect faces",anchor="w", font=("times new roman",20), bg="pink", fg="black",width="20",command=detect_face)
b3.place(x=0,y=340)

window.geometry("400x450")
window.mainloop()

<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\RITIK RAUNAK\AppData\Local\Temp\ipykernel_11716\3582185852.py:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
